[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fin-py/chatgpt-api-hands-on/blob/main/docs/chains.ipynb)

# Chains

chainはLLMとコンポーネントを組み合わせたアプリケーションを作成します。たとえば、入力されたデータをもとにテンプレートからフォーマットした結果をLLMに渡すchainを作成したります。

`PromptTemplate` で生成したpromptとOpenAIのLLMを組み合わせています。

In [1]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

llm = OpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["gender"],
    template="健やかに育ってくれるような {gender} の名前を考えてください?",
)
chain = LLMChain(llm=llm, prompt=prompt)
# Run the chain only specifying the input variable.
print(chain.run("男の子"))



・太郎
・拓真
・大翔
・亮太
・健太
・慎也
・優斗
・陸
・柊
・翼


次のコードでは `HumanMessagePromptTemplate` を使っています。

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)

human_message_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template="健やかに育ってくれるような {gender} の名前を考えてください?",
        input_variables=["gender"],
    )
)
chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])
chat = ChatOpenAI(temperature=0.9)
chain = LLMChain(llm=chat, prompt=chat_prompt_template)
print(chain.run("女の子"))

1. あかり (Akari)：「明るい」という意味で、陽気で明るく生き生きとした女の子にぴったりです。
2. かな (Kana)：「感謝」、「世話をする」という意味があり、優しく世話好きな女の子にぴったりです。
3. はるか (Haruka)：「遥かに」、「遠くへ」という意味があり、大きな夢を持って未来を見据える女の子にぴったりです。
4. ゆうか (Yuuka)：「優雅」、「優しさ」、「愛らしさ」という意味があり、優雅で美しい女の子にぴったりです。
5. さくら (Sakura)：「桜」という花の名前で、春に咲く美しい花をイメージさせる女の子にぴったりです。
6. あやか (Ayaka)：「彩果」という意味で、豊かな色彩を持つ果実をイメージさせる女の子にぴったりです。
7. みお (Mio)：「美緒」と書き、美しいものを愛する心優しい女の子にぴったりです。
8. ゆうき (Yuuki)：「勇気」、「勇敢さ」という意味があり、強い意志を持つ女の子にぴったりです。
9. まどか (Madoka)：「円」という意味があり、やさしく包み込むような女の子にぴったりです。
10. れいな (Reina)：「麗菜」と書き、美しく清楚な女の子にぴったりです。


`LLMChain` のインスタンスは実行（ `__call__` ）でき、引数 `inputs` に入力を渡せます。

In [3]:
chat = ChatOpenAI(temperature=0)
prompt_template = "{season} の俳句を読んでください"
llm_chain = LLMChain(
    llm=chat,
    prompt=PromptTemplate.from_template(prompt_template)
)
llm_chain(inputs={"season": "春"})

{'season': '春',
 'text': '春の風や\n馬車に揺られ\n花見かな\n\n春の雨や\n小川のせせらぎ\n心癒され\n\n春の夜や\n満月が照らす\n桜の花\n\n春の光や\n山々に映える\n新緑の葉\n\n春の風や\n鳥たちが舞う\n空の青'}

次のコードでは出力のみを返します。

In [4]:
llm_chain("夏", return_only_outputs=True)

{'text': '夏の風や\n蝉の声に\n涼を求む\n\n夏の夜や\n星が瞬く\n静寂の中\n\n夏の海や\n波の音に\n心癒され\n\n夏の日や\n蝶が舞う\n花の中'}

複数の入力がある場合には `run` メソッドの引数に辞書を渡します。

In [5]:
chat = ChatOpenAI(temperature=0)
prompt_template = "{theme} をテーマに {season} の俳句を読んでください"
llm_chain = LLMChain(
    llm=chat,
    prompt=PromptTemplate.from_template(prompt_template)
)
llm_chain.run({"theme": "旅行", "season": "秋"})

'紅葉の\n道を行く旅\n秋の風\n\n秋の旅\n山々の紅葉\n美しき\n\n秋の夜\n旅先の宿で\n月を仰ぐ\n\n旅先に\n秋の風が吹く\n草の香り\n\n秋の旅\n温泉に浸かり\n心癒す\n\n旅先で\n秋の味覚を\n堪能する'

## Add memory to chains

chainは `BaseMemory` オブジェクトをサポートしており、オブジェクトの呼び出しが複数あった際にデータを永続化できます。

In [6]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

conversation = ConversationChain(
    llm=chat,
    memory=ConversationBufferMemory()
)
conversation.run("干支の5番目までを教えてください")

'干支の5番目までは、子、丑、寅、卯、辰です。'

In [7]:
conversation.run("6番目以降を教えてください")

'6番目以降の干支は巳、午、未、申、酉、戌、亥です。'